# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,17.11,92,100,13.87,SH,1733103707
1,1,papatowai,-46.5619,169.4708,16.87,61,100,3.91,NZ,1733103709
2,2,ribeira grande,38.5167,-28.7000,17.10,90,49,1.34,PT,1733103710
3,3,adamstown,-25.0660,-130.1015,23.28,77,86,6.18,PN,1733103685
4,4,tiksi,71.6872,128.8694,-24.87,100,70,1.92,RU,1733103683


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng", "Lat",
    geo=True,
    size="Humidity",
    color="Humidity",
    tiles="OSM",
    frame_width=800,
    frame_height=500,
    title="City Humidity Map"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
min_temp = 20  # Minimum temperature in Celsius
max_temp = 30  # Maximum temperature in Celsius
max_humidity = 50  # Maximum humidity in percent
min_cloudiness = 0  # Minimum cloudiness in percent
max_cloudiness = 10  # Maximum cloudiness in percent

ideal_cities_df = city_data_df[
    (city_data_df["Max Temp"] >= min_temp) & 
    (city_data_df["Max Temp"] <= max_temp) & 
    (city_data_df["Humidity"] <= max_humidity) &
    (city_data_df["Cloudiness"] >= min_cloudiness) & 
    (city_data_df["Cloudiness"] <= max_cloudiness)
]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
184,184,maun,-19.9833,23.4167,20.98,32,0,1.54,BW,1733103788
229,229,villa hidalgo,21.6667,-102.6000,21.43,20,0,2.78,MX,1733104008
240,240,bondoukou,8.0333,-2.8000,22.76,35,0,0.91,CI,1733104022
252,252,dwarka,22.2394,68.9678,23.55,43,0,4.33,IN,1733104024
367,367,licheng,23.2955,113.8284,21.37,47,4,1.01,CN,1733104132
427,427,rakops,-21.0167,24.3333,24.91,18,0,4.54,BW,1733104271
432,432,kailua-kona,19.6406,-155.9956,28.28,49,0,3.60,US,1733104278
461,461,sao vicente,-23.9631,-46.3919,23.32,26,2,0.93,BR,1733104317
510,510,orapa,-21.2833,25.3667,22.55,26,0,4.33,BW,1733104384
546,546,tamale,9.4008,-0.8393,24.47,30,6,2.86,GH,1733104429


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
184,maun,BW,-19.9833,23.4167,32,
229,villa hidalgo,MX,21.6667,-102.6000,20,
240,bondoukou,CI,8.0333,-2.8000,35,
252,dwarka,IN,22.2394,68.9678,43,
367,licheng,CN,23.2955,113.8284,47,
427,rakops,BW,-21.0167,24.3333,18,
432,kailua-kona,US,19.6406,-155.9956,49,
461,sao vicente,BR,-23.9631,-46.3919,26,
510,orapa,BW,-21.2833,25.3667,26,
546,tamale,GH,9.4008,-0.8393,30,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 5000  # Radius in meters for hotel search
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found"
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
maun - nearest hotel: Center Lodge Conference Center
villa hidalgo - nearest hotel: No hotel found
bondoukou - nearest hotel: Hotel Akakro
dwarka - nearest hotel: The Dwarika Hotel
licheng - nearest hotel: No hotel found
rakops - nearest hotel: Rakops River Lodge
kailua-kona - nearest hotel: PACIFIC 19 Kona
sao vicente - nearest hotel: Pousada Vitória
orapa - nearest hotel: No hotel found
tamale - nearest hotel: Alhassan Hotel
tinogasta - nearest hotel: Hotel de Turismo


,City,Country,Lat,Lng,Humidity,Hotel Name
184,maun,BW,-19.9833,23.4167,32,Center Lodge Conference Center
229,villa hidalgo,MX,21.6667,-102.6000,20,No hotel found
240,bondoukou,CI,8.0333,-2.8000,35,Hotel Akakro
252,dwarka,IN,22.2394,68.9678,43,The Dwarika Hotel
367,licheng,CN,23.2955,113.8284,47,No hotel found
427,rakops,BW,-21.0167,24.3333,18,Rakops River Lodge
432,kailua-kona,US,19.6406,-155.9956,49,PACIFIC 19 Kona
461,sao vicente,BR,-23.9631,-46.3919,26,Pousada Vitória
510,orapa,BW,-21.2833,25.3667,26,No hotel found
546,tamale,GH,9.4008,-0.8393,30,Alhassan Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng", "Lat",
    geo=True,
    size="Humidity",
    color="Humidity",
    tiles="OSM",
    frame_width=800,
    frame_height=500,
    title="City Humidity Map with Hotels",
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hotel Name,Country)